In [135]:
# Install necessary packages
!pip install folium pandas

In [136]:
# Import packages
import folium
import pandas as pd

In [137]:
# Additional folium plugins
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [138]:
# Build dictionary to fill the dataframe

dictionary_1 = {'Name': ['Whiskey Island', 'Cleveland Pier', 'North Royalton Memorial Park', 'Cahoon Memorial Park East', 'Lakewood Park'],
                'Location': [ [41.499815281213294, -81.71402335033004], [41.51078251618141, -81.6965088525839], [41.310403887203094, -81.72745819483838], [41.48803473001595, -81.9239611312965], [41.496793619480506, -81.79561750008173] ],
                'Address': ['2800 Whiskey Island Dr, Cleveland, OH 44102', 'E 9th Street Pier, Cleveland, OH 44114', '14600 State Rd, North Royalton, OH 44133', '300 Bryson Ln, Bay Village, OH 44140', '1002 Parkside Dr, Lakewood, OH 44107'],
                'Sand_Or_Grass': ['Sand', 'Sand', 'Sand', 'Sand', 'Sand'],
                'By_Water': ['Y', 'Y', 'N', 'N', 'Y']
               }

# Convert dictionary to dataframe using Pandas

df = pd.DataFrame(dictionary_1)

In [141]:
# Take user's location in latitude and longitude

user_location = []

user_latitude = float(input("Input your location\'s latitude "))
user_longitude = float(input("Input your location\'s longitude "))

user_location = [user_latitude, user_longitude]

Input your location's latitude 41.48366744091054
Input your location's longitude -81.76792996833044


In [142]:
# Validate location is saved as a list
type(user_location)

list

In [143]:
# Plot user's location as starting point on the map
site_map = folium.Map(location=user_location, zoom_start = 15)

# Create marker for user's location
folium.Marker(
    user_location, 
    popup="You are here", 
    icon=folium.Icon(color="lightred", icon="user"),
    ).add_to(site_map)

In [144]:
# Create empty column to calculate the distance of the user from each court
df['Distance'] = None

In [145]:
# Install geopy to calculate distance to courts
!pip install geopy
import geopy.distance

In [146]:
# Calculates distance to 1 court and returns it
from math import radians, cos, sin, asin, sqrt

# distance definition is from: https://www.section.io/engineering-education/using-geopy-to-calculate-the-distance-between-two-points/#:~:text=February%2025%2C%202022&text=Geopy%20is%20a%20Python%20library,well%20as%20other%20data%20sources.
def distance_d(LaA, LoA, LaB, LoB):
# The function "radians" is found in the math module, It's also used to convert radians to degrees.  
    LoA = radians(LoA)  
    LoB = radians(LoB)  
    LaA= radians(LaA)  
    LaB = radians(LaB) 

    # The "Haversine formula" is used. This method was chosen for its high accuracy
    D_Lo = LoB - LoA 
    D_La = LaB - LaA 
    P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))
    
    # The earth's radius in kilometers.
    R_km = 6371  
    
    # Then we'll compute the outcome in km.
    distance_found = (Q * R_km)
    # Convert to miles
    distance_found_miles = distance_found * 0.621371
    # Formats distance to 2 decimal places
    formatted_distance_miles = round(distance_found_miles, 2)
    return formatted_distance_miles

In [147]:
# Use loop to calculate the distance to each court
for i in range(0,df.shape[0]):
    df.iloc[i,5] = distance_d(user_location[0], user_location[1], df.iloc[i,1][0], df.iloc[i,1][1])

In [148]:
# Plots all of the volleyball courts
for i in range(0,df.shape[0]):
    folium.Marker(
    df.iloc[i,1],
    popup = f"<div style='font-size: 12px; color: lightblue;'><b>{df.iloc[i, 0]}</b><br>{df.iloc[i,5]:.2f} mi</div>",
    icon=folium.Icon(color="lightblue", icon="info-circle"),
    ).add_to(site_map)
    
# Plots distance lines to all courts
for i in range(0,df.shape[0]):
    folium.PolyLine(
    locations = ([user_location[0],user_location[1]], [df.iloc[i,1][0], df.iloc[i,1][1]]), weight=2).add_to(site_map)

In [149]:
# Convert 'Distance' column to a numeric data type so that idxmin() will work
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

# Use idxmin() on the 'Distance' column to get index of closest court
min_index = df['Distance'].idxmin()

# Plot over the original marker with a new star marker for the closest court
folium.Marker(
    df.iloc[min_index,1],
    popup = f"<div style='font-size: 12px; color: lightblue;'><b>{df.iloc[min_index, 0]}</b><br>{df.iloc[min_index,5]:.2f} mi</div>",
    icon=folium.Icon(color="orange", icon="star"),
    ).add_to(site_map)

# Display map
site_map

Resulting ouput will look like this:
<img src="Cleveland_Volleyball_Location_Finder_Resulting_Map.jpg" style = "width:380px; height:324" title="Resulting ouput will look like this:">

In [134]:
# How to insert image to jupyter notebook: https://www.youtube.com/watch?v=1p0KfGTfsmI

'C:\\Users\\josep\\Cleveland Volleyball Locations'